In [1]:
#import libraries
import pandas as pd
import re
import requests
import time
from io import StringIO
import os
import json

In [ ]:
out_df = pd.read_csv('/Users/sanyakhan/Downloads/Daily_edition_parsed.csv', dtype=str)
out_df.head(5)
out_df = out_df

In [ ]:
members_fp = "/Users/sanyakhan/Downloads/Legislators_actual - legislators-historical.csv"
members_df = pd.read_csv(members_fp, dtype=str)
members_df.tail(5)
members_df.columns

In [ ]:

def norm_speech(x):
    if not isinstance(x, str): return None
    s = x
    if "house" in s.lower():  return "HOUSE"
    if "senate" in s.lower(): return "SENATE"
    return None

def norm_member(x):
    if not isinstance(x, str): return None
    s = x.strip().lower()
    if s in {"rep"}: return "HOUSE"
    if s in {"sen"}: return "SENATE"
    return None

def last_name(name):
    if not isinstance(name, str) or not name.strip():
        return []
    s = name.upper().strip()
    s = re.sub(r"[^\w\s'\-]", " ", s)
    s = re.sub(r"\s+", " ", s)
    parts = s.split(" ")
    full_last = " ".join(parts)
    return [full_last]

mem_df = pd.read_csv(members_fp, dtype=str)
mem_df.columns = [c.lower() for c in mem_df.columns]

mem_df["last_name_norm"] = mem_df["last_name"].fillna("").str.upper().str.strip()
mem_df["start_dt"] = pd.to_datetime(mem_df.get("term_start"), errors="coerce")
mem_df["end_dt"] = pd.to_datetime(mem_df.get("term_end"), errors="coerce").fillna(pd.Timestamp("2100-01-01"))

if "term_type" in mem_df.columns:
    mem_df["member_type"] = mem_df["term_type"].map(norm_member)
else:
    mem_df["member_type"] = pd.NA

by_full_last = {}
for i, r in mem_df.iterrows():
    key = r.get("last_name_norm", "")
    if isinstance(key, str) and key.strip():
        by_full_last.setdefault(key, []).append(i)

def pick_candidate(speaker, speech_date, speech_type):
    if not isinstance(speaker, str) or not speaker.strip():
        return None, None
    opts = last_name(speaker)
    full_last = opts[0] if opts else None
    if not full_last:
        return None, None
    cand_idx = set(by_full_last.get(full_last, []))
    if not cand_idx:
        return None, None
    cands = mem_df.loc[list(cand_idx)].copy()
    if isinstance(speech_type, str):
        strict = cands[(cands["member_type"].isna()) | (cands["member_type"] == speech_type)]
        if strict.empty:
            return None, None
    else:
        strict = cands
    if len(strict) == 1:
        r = strict.iloc[0]
        return str(r["party"]).strip(), str(r["state"]).strip()
    ps = strict[["party","state"]].dropna()
    if not ps.empty and len(ps.drop_duplicates()) == 1:
        r = ps.iloc[0]
        return str(r["party"]).strip(), str(r["state"]).strip()
    return None, None

out_df["speech_type"]= out_df["Chamber"].map(norm_speech) if "Chamber" in out_df.columns else None
out_df["Date"] = pd.to_datetime(out_df["Date"], errors="coerce")
mask_to_fix = out_df["Party"].isna() | out_df["State"].isna()
filled_party = filled_state = 0

for i in out_df.index[mask_to_fix]:
    party0, state0 = out_df.at[i,"Party"], out_df.at[i,"State"]
    party, state = pick_candidate(out_df.at[i,"Speaker"], out_df.at[i,"Date"], out_df.at[i,"speech_type"])
    if pd.isna(party0) and party:
        out_df.at[i,"Party"] = party
        filled_party += 1
    if pd.isna(state0) and state:
        out_df.at[i,"State"] = state
        filled_state += 1

out_df.drop(columns=["speech_type"], errors="ignore", inplace=True)
print(f"Filled Party: {filled_party} | Filled State: {filled_state}")


In [ ]:
out_df

In [ ]:
na_indexes = out_df.index[out_df["Party"].isna()].tolist()
index = na_indexes
len(index)

In [ ]:
for each in index:
    curr_name = out_df['Speaker'][each]
    date = out_df['Date'][each]
    #print('Current Name: ', curr_name)
    #print('Speech Date: ', date)
    #print('-----')
    temp_list = []
    #for courtney current go through whole DS
    for i in members_df.index:
        #Have i accounted for edge cases?
        if members_df['last_name'][i] == curr_name:
            if pd.to_datetime(members_df['term_start'][i]) <= pd.to_datetime(date) <= pd.to_datetime(members_df['term_end'][i]):
                 #print(members_df['first_name'][i] + ' ' + members_df['last_name'][i])
                 temp_list.extend([i]) 
    #print(temp_list)
    if len(temp_list) == 1:
        out_df['State'][each] = members_df['state'][temp_list[0]]
        out_df['Party'][each] = members_df['party'][temp_list[0]]       


In [ ]:
out_df
na_indexes = out_df.index[out_df["Party"].isna()].tolist()
len(na_indexes)